In [90]:
""" %pip install matplotlib
%pip install plotly
%pip install pandas
%pip install numpy
%pip install datetime"""

import re
import numpy as np
import pandas as pd
import datetime as dt
from datetime import datetime
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
import plotly.express as px

In [91]:
def clearvars():    
    for el in sorted(globals()):
        if '__' not in el:
                print(f'deleted: {el}')
                del el
clearvars()

deleted: In
deleted: Out
deleted: _
deleted: _21
deleted: _26
deleted: _27
deleted: _37
deleted: _38
deleted: _4
deleted: _45
deleted: _46
deleted: _5
deleted: _53
deleted: _54
deleted: _61
deleted: _62
deleted: _69
deleted: _7
deleted: _70
deleted: _85
deleted: _86
deleted: _9
deleted: _VSCODE_rwho_ls
deleted: _dh
deleted: _i
deleted: _i1
deleted: _i10
deleted: _i11
deleted: _i12
deleted: _i13
deleted: _i14
deleted: _i15
deleted: _i16
deleted: _i17
deleted: _i18
deleted: _i19
deleted: _i2
deleted: _i20
deleted: _i21
deleted: _i22
deleted: _i23
deleted: _i24
deleted: _i25
deleted: _i26
deleted: _i27
deleted: _i28
deleted: _i29
deleted: _i3
deleted: _i30
deleted: _i31
deleted: _i32
deleted: _i33
deleted: _i34
deleted: _i35
deleted: _i36
deleted: _i37
deleted: _i38
deleted: _i39
deleted: _i4
deleted: _i40
deleted: _i41
deleted: _i42
deleted: _i43
deleted: _i44
deleted: _i45
deleted: _i46
deleted: _i47
deleted: _i48
deleted: _i49
deleted: _i5
deleted: _i50
deleted: _i51
deleted: _i52
dele

In [92]:
base_path = 'dataRaw/'
dataRaw = pd.read_csv(base_path + 'data.csv', sep=',', low_memory=False)
holidays_events = pd.read_csv(base_path + 'holidays_events(India).csv', sep=';')
items = pd.read_csv(base_path + 'items.csv', sep=';', low_memory=False)
oilRaw = pd.read_csv(base_path + 'oil(India).csv', sep=',', skipinitialspace=True)
stores = pd.read_csv(base_path + 'stores.csv', sep=';', encoding='ISO-8859-1')
testRaw = pd.read_csv(base_path + 'test.csv', sep=',')
transactions = pd.read_csv(base_path + 'transactions.csv', sep=',')


In [93]:
test = testRaw.copy()
test['date'] = pd.to_datetime(test['date'], format='%Y-%m-%d')

# --------------------------------------------------------------------------------------------

""" test.to_csv('data/test.csv', index=False)
> git push origin main:main
remote: error: Trace: 25213f19d457fb2989564964b10ab27c2fb682233e594fb81ff2ed78998776c2        
remote: error: See https://gh.io/lfs for more information.        
remote: error: File data/test.csv is 120.32 MB; this exceeds GitHub's file size limit of 100.00 MB        
remote: error: GH001: Large files detected. You may want to try Git Large File Storage - https://git-lfs.github.com.        
To https://github.com/florboydens/Data-Analytics-Case-2.git
 ! [remote rejected] main -> main (pre-receive hook declined)
error: failed to push some refs to 'https://github.com/florboydens/Data-Analytics-Case-2.git' """

" test.to_csv('data/test.csv', index=False)\n> git push origin main:main\nremote: error: Trace: 25213f19d457fb2989564964b10ab27c2fb682233e594fb81ff2ed78998776c2        \nremote: error: See https://gh.io/lfs for more information.        \nremote: error: File data/test.csv is 120.32 MB; this exceeds GitHub's file size limit of 100.00 MB        \nremote: error: GH001: Large files detected. You may want to try Git Large File Storage - https://git-lfs.github.com.        \nTo https://github.com/florboydens/Data-Analytics-Case-2.git\n ! [remote rejected] main -> main (pre-receive hook declined)\nerror: failed to push some refs to 'https://github.com/florboydens/Data-Analytics-Case-2.git' "

In [94]:
data = dataRaw.copy()
data['date'] = pd.to_datetime(dataRaw['date'], format='%Y-%m-%d')
data['unit_sales'] = dataRaw['unit_sales'].round(2)
data['onpromotion'].fillna(False, inplace=True)

# --------------------------------------------------------------------------------------------

""" 
data.to_csv('data/data.csv', index=False)
...
Zelfde reden als de Test file, te groot
"""

C:\Users\Cursist\AppData\Local\Temp\ipykernel_16120\705338131.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['onpromotion'].fillna(False, inplace=True)
C:\Users\Cursist\AppData\Local\Temp\ipykernel_16120\705338131.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['onpromotion'].fillna(False, in

" \ndata.to_csv('data/data.csv', index=False)\n...\nZelfde reden als de Test file, te groot\n"

In [95]:
mask = oilRaw['dcoilwtico;;'].str.contains('%')
oilExtra = oilRaw[mask].copy()
oil = oilRaw[~mask].copy()
oil.reset_index(drop=True, inplace=True)
oil['dcoilwtico;;'] = oil['dcoilwtico;;'].str.replace(';', '')
oil['dcoilwtico;;'] = oil['dcoilwtico;;'].apply(lambda x: x.strip())
oil = oil[~(oil['dcoilwtico;;'] == '')]
try:
    oil['dcoilwtico;;'] = oil['dcoilwtico;;'].astype(float).round(2)
except:
    oil.dropna(subset=['dcoilwtico;;'], inplace=True)
oil.rename(columns={'dcoilwtico;;': 'dcoilwtico'}, inplace=True)

# --------------------------------------------------------------------------------------------

oil.to_csv('data/oil.csv', index=False)
oilExtra.to_csv('data/oilExtra.csv', index=False)


In [96]:
transactions['date'] = pd.to_datetime(test['date'], format='%Y-%m-%d')
transactions['month_day'] = transactions['date'].dt.strftime('%m-%d')

# --------------------------------------------------------------------------------------------

transactions.to_csv('data/transactions.csv', index=False)

In [ ]:
holidays_events = pd.read_csv(base_path + 'holidays_events(India).csv', sep=';')

holidays_events = holidays_events.drop(columns=['type'])
holidays_events.dropna(subset=['date'], inplace=True)
holidays_events['date'] = holidays_events['date'].str.replace(' ', '/')
holidays_events['date'] = holidays_events['date'].str.replace('mrt', 'mar')
holidays_events['date'] = holidays_events['date'].str.replace('mei', 'may')
holidays_events['date'] = holidays_events['date'].str.replace('okt', 'oct')
holidays_events = holidays_events.drop_duplicates()
holidays_events['date'] = pd.to_datetime(holidays_events['date'], format='%d/%b')   #.dt.strftime('%Y-%m-%d')
# if the date is the same from earlier in the dataset, drop the row.
holidays_events = holidays_events[~holidays_events['date'].duplicated()]
holidays_events['month_day'] = holidays_events['date'].dt.strftime('%m-%d')
holidays_events = holidays_events.sort_values('date')
holidays_events = holidays_events.reindex(columns=['date', 'month_day', 'description', 'transferred'])
holidays_events.rename(columns={'transferred': 'type'}, inplace=True)

# --------------------------------------------------------------------------------------------

holidays_events.to_csv('data/holidays_events.csv', index=False)